In [1]:
# Module Importations


In [4]:
# Project Module Importations
from GensetSimulation.data_analytics.src.data import cosmos_client_manager

In [3]:
# Constants (i.e. for saving figures)

In [6]:
# Load data from database
client = cosmos_client_manager.intialise_client()